**Script greenspace data frame**

1. Reading a raster file

In [41]:
import pandas as pd
import scipy as sp
from scipy.ndimage import convolve
import numpy as np
import rasterio
from rasterio.transform import from_origin
import os
import geopandas as gpd
from osgeo import gdal
%matplotlib inline

# Step 1: Get input NDVI
# data dir
data_dir = r"thesis_project/data"
fp = os.path.join(data_dir, "CR_NDVI.TIF")

# open the file:
ndvi_input_raster = rasterio.open(fp)

# check type of the variable 'raster'
type(ndvi_input_raster)

rasterio.io.DatasetReader

In [44]:
# All Metadata for the whole raster dataset
ndvi_input_raster.meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': -3.3999999521443642e+38,
 'width': 7443,
 'height': 5987,
 'count': 1,
 'crs': CRS.from_epsg(27700),
 'transform': Affine(10.0, 0.0, 341660.6724,
        0.0, -10.0, 431034.6389)}

In [42]:
# Read the raster band as separate variable
ndvi_input = ndvi_input_raster.read(1)
type(ndvi_input)

numpy.ndarray

In [23]:
# check shape of the raster image
ndvi_input.shape

(5987, 7443)

2. Apply focal statistics

In [24]:
# design kernel at a 100m scale
scale_100 = np.ones((21, 21))
#scale_100
scale_100.shape

numpy.ndarray

In [25]:
# convolve NDVI input with kernel at 100m (equivalent to average operation)
ndvi_output_100 = convolve(ndvi_input, scale_100) / np.sum(scale_100) # mode=reflect to extend input beyond its boundaries
ndvi_output_100.shape

(5987, 7443)

3. Write scaled greenspace layer to a raster image

In [63]:
# convert NDVI output to raster image
transform = from_origin(341660.6724, 431034.6389, 10.0, -10.0)

new_dataset = rasterio.open('thesis_project/data/output/ndvi_output_100.tif', 'w', driver='GTiff',
                            height = ndvi_output_100.shape[0], width = ndvi_output_100.shape[1],
                            count=1, dtype=str(ndvi_output_100.dtype),
                            crs='+proj=tmerc +lat_0=49 +lon_0=-2 +k=0.9996012717 +x_0=400000 +y_0=-100000 +ellps=airy +datum=OSGB36 +units=m +no_defs',
                            transform=transform)

new_dataset.write(ndvi_output_100, 1)
new_dataset.close()

In [70]:
# check ndvi output raster
#new_dataset.meta
type(new_dataset)

rasterio.io.DatasetWriter

4. Extract raster values at point locations

In [67]:
# Step 1: Get input %Greenness
# data dir
#data_dir = r"thesis_project/data"
fp_green = os.path.join(data_dir, "GreenNoGreenRes.TIF")

# open the file:
green_input_raster = rasterio.open(fp_green)

# check type of the variable 'raster'
type(green_input_raster)

rasterio.io.DatasetReader

In [69]:
# All Metadata for the whole raster dataset
green_input_raster.meta

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': 255.0,
 'width': 16000,
 'height': 12000,
 'count': 1,
 'crs': CRS.from_epsg(27700),
 'transform': Affine(5.0, 0.0, 339999.2458,
        0.0, -5.0, 430003.9122)}

In [56]:
# create NDVI layers at three different scales (100m, 300m, and 500m)
scales = [21, 61, 101]
#scales = [21]

#ndvi_outputs = []
greenspace_dfs = []

for scale in scales:
    
    kernel = np.ones((scale, scale)) # build kernel 
    ndvi_output = (convolve(ndvi_input_arr, kernel) / np.sum(kernel)).ravel() # convert to 1d array
    greenspace_df = pd.DataFrame(ndvi_output, columns=['NDVI']) # create df
    greenspace_dfs.append(greenspace_df) # append to df list

#    ndvi_outputs.append(ndvi_output)
    
#print(len(ndvi_outputs))
print(len(greenspace_dfs))

3


In [57]:
# data frame NDVI at 100m
greenspace_df_100 = greenspace_dfs[0]
greenspace_df_100.head()

,NDVI
0,0.754154
1,0.754687
2,0.755890
3,0.757124
4,0.756698


In [58]:
# data frame NDVI at 300m
greenspace_df_300 = greenspace_dfs[1]
greenspace_df_300.head()

,NDVI
0,0.747358
1,0.747058
2,0.746479
3,0.745983
4,0.745491


In [59]:
# data frame NDVI at 500m
greenspace_df_500 = greenspace_dfs[2]
greenspace_df_500.head()

,NDVI
0,0.668690
1,0.668646
2,0.668888
3,0.669157
4,0.669527


In [53]:
ndvi_output_100.shape

(44561241,)

In [33]:
# check sum of kernel (denominator of average operation)
np.sum(scale_100)

441.0

In [34]:
# output map of averaged neighbour
#output_map_100
ndvi_output_100.shape

(5987, 7443)

**DRAFT**

In [48]:
ndvi_output_100_1d = ndvi_output_100.ravel()
ndvi_output_100_1d.shape

(44561241,)

In [36]:
# randomly sample 1000 points from 2d array
np.random.seed(1234) #set seed
ndvi_points_100 = np.random.choice(ndvi_output_100.ravel(), 1000 ,replace=False)
ndvi_points_100

array([ 0.5321067 ,  0.49312577,  0.6024407 ,  0.46361962,  0.5726831 ,
        0.51598865,  0.73149955,  0.37032697,  0.5522508 ,  0.50234264,
        0.65675926,  0.43822482,  0.68798816,  0.55864656,  0.5882455 ,
        0.15388723,  0.3068263 ,  0.5331158 ,  0.70567346,  0.6178858 ,
        0.6112739 ,  0.6388109 ,  0.48111692,  0.43576893,  0.56005317,
        0.2435806 ,  0.7249661 ,  0.41961008,  0.49273118,  0.7204542 ,
        0.55228615,  0.7226849 ,  0.72932744,  0.5088997 ,  0.4124835 ,
        0.23340541,  0.41825143,  0.542702  ,  0.37826246,  0.52238214,
        0.5915935 ,  0.65983987,  0.5584354 ,  0.72806144,  0.44634473,
        0.7342117 ,  0.63798326,  0.30763182,  0.45787397,  0.36749846,
        0.5860451 ,  0.5985585 ,  0.63174736,  0.39039016,  0.30227262,
        0.15829873,  0.3264552 ,  0.58864754,  0.39561763,  0.6275784 ,
        0.660751  ,  0.77967143,  0.16108127,  0.6841988 ,  0.6677631 ,
        0.32129216,  0.3798714 ,  0.73147833,  0.40116498,  0.34

5. Store NDVI values for 1000 points into a df

In [37]:
# creating a df from array
greenspace_metrics_100 = pd.DataFrame(ndvi_points_100, columns=['NDVI'])
#greenspace_metrics_100.head()
greenspace_metrics_100

,NDVI
0,0.532107
1,0.493126
2,0.602441
3,0.463620
4,0.572683
...,...
995,0.545768
996,0.581678
997,0.228566
998,0.615783


Here, repeat again the same process but with %greenness

6. Read %Greenness input file

In [25]:
# reading raster file using gdal
green_input_map = gdal.Open("thesis_project/data/GreenNoGreenRes.tif")

7. Applying focal statistics to %greenness input map at 100m scale

In [38]:
# convert input map to array
green_input_arr = np.array(green_input_map.GetRasterBand(1).ReadAsArray())
#green_input_arr
green_input_arr.shape

(12000, 16000)

In [23]:
# fix kernel shape to 100m scale (21x21)
scale_100 = np.ones((21, 21))
#scale_100
scale_100.shape

(21, 21)

In [27]:
# convolve greenness input map with kernel 100m (equivalent to percentage operation)
green_output_100 = convolve(green_input_arr, scale_100) / np.sum(scale_100) # mode=reflect to extend input beyond its boundaries


8. Output layer averaged at 100m scale

In [39]:
# greenness output map after performing focal statistics
green_output_100
#green_output_100.shape

array([[0.41723356, 0.41496599, 0.41269841, ..., 0.3106576 , 0.30839002,
        0.30839002],
       [0.41496599, 0.41043084, 0.40589569, ..., 0.30385488, 0.29931973,
        0.29931973],
       [0.3968254 , 0.39455782, 0.39229025, ..., 0.3106576 , 0.30385488,
        0.3015873 ],
       ...,
       [0.41950113, 0.41950113, 0.41950113, ..., 0.41950113, 0.41950113,
        0.41950113],
       [0.41950113, 0.41950113, 0.41950113, ..., 0.41950113, 0.41950113,
        0.41950113],
       [0.41950113, 0.41950113, 0.41950113, ..., 0.41950113, 0.41950113,
        0.41950113]])

10. Store %greenness values for 1000 points into a df

In [41]:
# adding values to data frame
greenspace_metrics_100['%Greenness'] = green_points_100
#greenspace_metrics_100.head()
greenspace_metrics_100

,NDVI,%Greenness
0,0.532107,0.419501
1,0.493126,0.000000
2,0.602441,0.419501
3,0.463620,0.419501
4,0.572683,0.356009
...,...,...
995,0.545768,0.419501
996,0.581678,0.419501
997,0.228566,0.419501
998,0.615783,0.410431


9. Sample 1000 points and extract %greenness values

In [40]:
# randomly sample 1000 points
np.random.seed(2345) #set seed
green_points_100 = np.random.choice(green_output_100.ravel(), 1000 ,replace=False)
green_points_100

array([0.41950113, 0.        , 0.41950113, 0.41950113, 0.35600907,
       0.21315193, 0.41950113, 0.11111111, 0.51927438, 0.41950113,
       0.47845805, 0.41950113, 0.0861678 , 0.41950113, 0.33333333,
       0.41950113, 0.41950113, 0.41950113, 0.41950113, 0.13378685,
       0.41950113, 0.41950113, 0.41950113, 0.41950113, 0.3015873 ,
       0.41950113, 0.41950113, 0.41950113, 0.41950113, 0.41950113,
       0.26077098, 0.41950113, 0.33786848, 0.41950113, 0.41950113,
       0.41950113, 0.4399093 , 0.29705215, 0.41950113, 0.37414966,
       0.41950113, 0.41950113, 0.26303855, 0.41950113, 0.41950113,
       0.41950113, 0.22675737, 0.41950113, 0.41950113, 0.41950113,
       0.1292517 , 0.41950113, 0.20408163, 0.41950113, 0.41950113,
       0.41950113, 0.38548753, 0.25623583, 0.12471655, 0.41043084,
       0.31972789, 0.41950113, 0.11111111, 0.41950113, 0.39229025,
       0.41950113, 0.38321995, 0.41950113, 0.41269841, 0.41950113,
       0.41950113, 0.41950113, 0.40589569, 0.06122449, 0.41950